In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [4]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
    'H-UNet-URPC-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_urpc_swta_t')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,100,37.59,23.14
1,H-UNet-SWTA-T,0,5,100,38.85,24.11
2,H-UNet-SWTA-T,0,10,100,40.69,25.55
3,H-UNet-SWTA-T,0,20,100,41.58,26.25
4,H-UNet-SWTA-T,0,50,100,41.81,26.43
5,H-UNet-SWTA-T,0,75,100,41.85,26.46
6,H-UNet-SWTA-T,0,100,100,41.87,26.48
7,H-UNet-URPC-SWTA-T,0,1,100,61.08,43.97
8,H-UNet-URPC-SWTA-T,0,5,100,61.66,44.57
9,H-UNet-URPC-SWTA-T,0,10,100,61.22,44.12


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                     Mean CI 0.9%    Mean CI 0.9%
Model              Inv Temp Regime                               
H-UNet-SWTA-T      1        100     37.59     NaN   23.14     NaN
                   5        100     38.85     NaN   24.11     NaN
                   10       100     40.69     NaN   25.55     NaN
                   20       100     41.58     NaN   26.25     NaN
                   50       100     41.81     NaN   26.43     NaN
                   75       100     41.85     NaN   26.46     NaN
                   100      100     41.87     NaN   26.48     NaN
H-UNet-URPC-SWTA-T 1        100     61.08     NaN   43.97     NaN
                   5        100     61.66     NaN   44.57     NaN
                   10       100     61.22     NaN   44.12     NaN
                   20       100     60.84     NaN   43.72     NaN
                   50       100     60.74     NaN   43.61     NaN
                   75       100     60.73     NaN   43.60     NaN
                   100      100     60.72     NaN   43.60     NaN

In [47]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
    #'H-EM-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_em_unet_swta_t')),
    #'H-UAMT-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_uamt_unet_swta_t')),
    'H-CPS-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_cps_unet_swta_t')),
    #'H-URPC-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_urpc_unet_swta_t')),
    #'H-CCT-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_cct_unet_swta_t')),
}

In [48]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [49]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-CPS-SWTA-T,0,1,1,70.02,53.87
1,H-CPS-SWTA-T,0,1,2,72.86,57.31
2,H-CPS-SWTA-T,0,1,5,75.53,60.68
3,H-CPS-SWTA-T,0,1,10,79.70,66.25
4,H-CPS-SWTA-T,0,1,20,82.04,69.54
...,...,...,...,...,...,...
345,H-CPS-SWTA-T,9,100,1,70.78,54.77
346,H-CPS-SWTA-T,9,100,2,67.80,51.28
347,H-CPS-SWTA-T,9,100,5,76.25,61.62
348,H-CPS-SWTA-T,9,100,10,78.02,63.96


Dice           Jaccard          
                                Mean   CI 0.9%    Mean   CI 0.9%
Model        Inv Temp Regime                                    
H-CPS-SWTA-T 1        1       70.470  1.232086  54.446  1.496717
                      2       71.788  1.232498  56.033  1.487767
                      5       76.313  1.075321  61.732  1.412168
                      10      80.614  0.906819  67.550  1.272341
                      20      83.664  1.112581  71.956  1.631924
             5        1       70.772  1.217468  54.805  1.476033
                      2       72.227  1.697275  56.610  2.081398
                      5       76.529  1.148563  62.025  1.515050
                      10      80.168  0.794535  66.919  1.109706
                      20      83.792  1.128569  72.152  1.665528
             10       1       70.861  1.235699  54.914  1.502315
                      2       71.873  1.323223  56.143  1.610327
                      5       76.764  1.255997  62.338  1.677647
                      10      80.631  0.957480  67.578  1.342038
                      20      83.577  0.951864  71.819  1.401137
             20       1       70.447  1.275689  54.421  1.543408
                      2       71.997  1.313479  56.294  1.587629
                      5       76.707  1.358279  62.273  1.813176
                      10      81.102  1.276754  68.268  1.812198
                      20      83.818  0.916908  72.171  1.356098
             50       1       70.874  1.270746  54.934  1.550086
                      2       71.347  1.252273  55.498  1.506772
                      5       76.732  1.039331  62.284  1.384546
                      10      81.009  1.149251  68.123  1.631847
                      20      84.025  1.061220  72.494  1.566322
             75       1       70.511  1.266495  54.499  1.529797
                      2       71.693  1.316341  55.922  1.588904
                      5       76.458  1.315832  61.941  1.744098
                      10      81.408  0.880283  68.672  1.241598
                      20      84.527  0.920135  73.229  1.373809
             100      1       70.657  1.126996  54.662  1.364427
                      2       71.254  1.173456  55.382  1.407090
                      5       77.039  1.182585  62.696  1.572435
                      10      81.094  1.184523  68.250  1.679592
                      20      84.196  0.979875  72.739  1.444638

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [7]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
    'CPS': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cps_unet')),
    'URPC': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('urpc_unet')),
    'CCT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cct_unet')),
}

In [8]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [9]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,CCT,0,1,1,67.77,51.25
1,CCT,0,1,2,70.38,54.29
2,CCT,0,1,5,74.36,59.19
3,CCT,0,1,10,81.15,68.27
4,CCT,0,1,20,82.44,70.12
...,...,...,...,...,...,...
295,URPC,9,1,1,69.42,53.16
296,URPC,9,1,2,67.94,51.44
297,URPC,9,1,5,73.03,57.52
298,URPC,9,1,10,78.28,64.31


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
CCT   1        1       69.508  1.127252  53.300  1.352967
               2       70.437  1.068993  54.395  1.278210
               5       76.722  1.187579  62.279  1.559367
               10      81.351  0.915017  68.590  1.283394
               20      84.936  0.875354  73.843  1.318532
CPS   1        1       70.001  0.916001  53.869  1.093109
               2       71.559  1.411084  55.767  1.718967
               5       76.543  1.090913  62.039  1.434285
               10      81.421  0.792441  68.683  1.116813
               20      84.449  0.764957  73.105  1.156714
EM    1        1       69.550  1.278488  53.360  1.541775
               2       70.791  1.531235  54.852  1.857711
               5       75.739  1.073473  60.984  1.389841
               10      78.292  0.821111  64.348  1.102924
               20      81.486  0.864500  68.783  1.229583
UAMT  1        1       69.654  1.193144  53.476  1.433769
               2       70.251  1.460897  54.201  1.766214
               5       75.576  0.951420  60.767  1.242891
               10      79.481  0.746279  65.965  1.013731
               20      83.269  0.741795  71.355  1.087672
UNet  1        1       68.469  0.663928  52.065  0.784732
               2       70.532  1.267591  54.521  1.523118
               5       76.499  1.264902  61.991  1.668055
               10      80.832  1.090293  67.869  1.537295
               20      84.509  0.485512  73.179  0.730902
URPC  1        1       68.618  0.563173  52.235  0.656471
               2       69.324  1.427276  53.104  1.724666
               5       74.747  1.191352  59.719  1.526734
               10      79.113  1.529320  65.520  2.077778
               20      82.635  1.909080  70.537  2.622215